# GenAI Prompt Engineering Examples

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [1]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.secretmanager', 'google-cloud-secret-manager'),
    ('google.genai', 'google-genai')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [2]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [3]:
REGION = 'us-central1'

# Set the BUCKET name for saving work:
BUCKET = PROJECT_ID
SECRET_ID = "google-aistudio" # for using Google AI Studio
SECRET_VERSION_ID = 1

packages:

In [4]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import secretmanager

import vertexai.vision_models # Imagen Models
import vertexai.preview.vision_models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

from google import genai # new unified SDK
from google.genai import types

import json
import io
import base64
import asyncio
import requests
import IPython
import datetime, time

In [5]:
print("Vertex AI version: " + str(aiplatform.__version__))

Vertex AI version: 1.74.0


clients:

In [6]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

bucket = gcs.lookup_bucket(BUCKET)

In [7]:
prompt = "What is a mulligan?"

---
## Vertex AI SDK

### Model list

In [20]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-002")
gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision-001")
gemini15_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-002")
gemini15_flash = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-002")
gemini2_flash = vertexai.generative_models.GenerativeModel("gemini-2.0-flash-exp") # new experimental model

# PaLM and Codey Models
text_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison')
chat_model = vertexai.language_models.ChatModel.from_pretrained('chat-bison')
textembed_model = vertexai.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison')
codecomp_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-gecko')
codechat_model = vertexai.language_models.CodeChatModel.from_pretrained('codechat-bison')

# Imagen Models
imagecap_model = vertexai.vision_models.ImageCaptioningModel.from_pretrained("imagetext")
imageqna_model = vertexai.vision_models.ImageQnAModel.from_pretrained("imagetext")
imagetext_model = vertexai.vision_models.ImageTextModel.from_pretrained("imagetext")
multimodalembed_model = vertexai.vision_models.MultiModalEmbeddingModel.from_pretrained('multimodalembedding')
imagen1 = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@002')
imagen2 = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@005')
imagen3 = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagen-3.0-generate-001')

### Text Prompt

In [9]:
response = gemini2_flash.generate_content(prompt)
#response

In [10]:
print(response.text)

The term "mulligan" has a few different meanings, but it most commonly refers to a second chance or redo, typically in a game. Here's a breakdown of the most common uses:

**1. In Golf:**

* **The Classic Mulligan:** In golf, a mulligan is an informal rule (not part of the official rules of golf) that allows a player to replay a shot, typically on their first tee shot. It's essentially a "do-over" for a bad initial attempt.
* **Casual Golf Only:** Mulligans are almost exclusively used in casual or friendly rounds of golf. They are never allowed in tournaments or competitive play.
* **A Gentleman's Agreement:** The use of mulligans is usually a mutual agreement between the players involved.
* **Different Interpretations:** There can be variations in how mulligans are used. Some groups allow only one per round, some allow them only on the first tee, and others have looser rules.

**2. In Other Games (Less Common but Similar Concept):**

* **Card Games (Hand Re-deal):** In some card games

In [11]:
IPython.display.Markdown(response.text)

The term "mulligan" has a few different meanings, but it most commonly refers to a second chance or redo, typically in a game. Here's a breakdown of the most common uses:

**1. In Golf:**

* **The Classic Mulligan:** In golf, a mulligan is an informal rule (not part of the official rules of golf) that allows a player to replay a shot, typically on their first tee shot. It's essentially a "do-over" for a bad initial attempt.
* **Casual Golf Only:** Mulligans are almost exclusively used in casual or friendly rounds of golf. They are never allowed in tournaments or competitive play.
* **A Gentleman's Agreement:** The use of mulligans is usually a mutual agreement between the players involved.
* **Different Interpretations:** There can be variations in how mulligans are used. Some groups allow only one per round, some allow them only on the first tee, and others have looser rules.

**2. In Other Games (Less Common but Similar Concept):**

* **Card Games (Hand Re-deal):** In some card games, like poker or Magic: The Gathering, a "mulligan" might refer to the option to discard your starting hand and receive a new one, often with some penalty (like drawing fewer cards in the new hand).
* **General "Do-Over":** More broadly, "mulligan" can sometimes be used informally to mean any kind of redo or second chance in other games or situations, even outside of golf.

**3. In General Conversation:**

* **Informal "Second Chance":** In everyday conversations, you might hear someone say they want a "mulligan" when they've made a mistake or want to try something again. It carries the same connotation of wanting a do-over.

**Key Takeaways:**

* **Most common use is in golf:** This is where the term originated and is most commonly recognized.
* **Informal and Casual:** Mulligans are typically used in casual settings and are not part of official rules.
* **A "Do-Over":** The core concept is that it's a second chance to correct a mistake.
* **Context is Important:** The specific rules and usage of a mulligan can vary depending on the context.

In summary, a mulligan is generally a second chance, most notably in golf, where it's a do-over of a bad shot, usually the first tee shot. It implies a more relaxed and casual approach to whatever is being played.


### Streaming Response

In [12]:
for response in gemini2_flash.generate_content(prompt, stream = True):
    print(response.text)

The
 term "mulligan" generally refers to **a second chance to perform an
 action, typically in a game, after a flawed or unsatisfactory first attempt.** It
's most commonly used in golf, but it can also appear in other contexts.

Here's a more detailed breakdown of what a mulligan means:


**In Golf:**

* **Definition:** In golf, a mulligan is an informal rule that allows a player to replay a shot, usually the first
 shot of a round or a tee shot, without incurring a penalty.
* **Informal Nature:** Mulligans are not recognized by the official rules of golf. They are purely a friendly agreement among players.
* **Common Usage:**
 Mulligans are often used in casual, non-competitive rounds of golf, especially among friends. They're meant to be a forgiving way to start the game.
* **Variations:** Some groups might allow a mulligan only on
 the first tee, while others might allow a limited number of mulligans per round, or even on specific holes.
* **Etymology (Origin):** The origin of the 

### Async Response

The client has built in method for awaitable responses that make it easy to make asynchronous request.

> For detailed coverage and examples of asynchronous call to these API's, scaling, error handling, and managing fail over regions check out this notebook in the same folder: [Python Asynchronous API Calls](./Python%20Asynchronous%20API%20Calls.ipynb)

In [16]:
response = await gemini_text.generate_content_async(prompts[1])
#print(response.text)
IPython.display.Markdown(response.text)

## Unplayable Lies in Golf

An unplayable lie in golf can be frustrating, but don't worry, there are options! Here's what you can do:

**1. Take a Penalty Stroke:**

This is the simplest option. You can take a one-stroke penalty and drop the ball within two club lengths of the original spot, no closer to the hole. Keep in mind that these two club lengths should be measured in any direction, except towards the hole.

**2. Play the Ball as it Lies:**

While challenging, you can attempt to play the ball as it lies. This could be a good option if you're confident in your ability to hit a difficult shot or if the penalty stroke would put you in an even worse position.

**3. Take Lateral Relief:**

If your ball is in a penalty area (water hazard or out-of-bounds), you can take lateral relief. This means dropping the ball within two club lengths of the point where the ball crossed the margin of the penalty area, but not closer to the hole. There is a two-stroke penalty for taking lateral relief.

**4. Provisional Ball:**

Before hitting your original shot, you can hit a provisional ball in case your original shot ends up being unplayable. You can then play the provisional ball without penalty if your original shot is deemed unplayable.

**Additional Considerations:**

* Always check the local rules before determining how to proceed with an unplayable lie.
* Consider the conditions and your own skill level when choosing the best option for your situation.
* Remember to inform your playing partners and marker when taking relief or playing a provisional ball.

Here are some helpful resources:

* **USGA Rule 19:** https://www.usga.org/content/dam/usga/pdf/2023/rule-books/rules-of-golf.pdf
* **R&A Rule 19:** https://www.randa.org/en/rules-and-amateur-status/rules-of-golf
* **PGA Instruction on Unplayable Lie:** https://www.pga.com/practice/golf-tips/how-to-play-an-unplayable-lie

I hope this helps! Let me know if you have any other questions about golf or anything else.

---
## Google GenAI SDK

In [13]:
# Create the Secret Manager client.
secret_client = secretmanager.SecretManagerServiceClient()
name = f"projects/{PROJECT_ID}/secrets/{SECRET_ID}/versions/{SECRET_VERSION_ID}"
response = secret_client.access_secret_version(request={"name": name})
aistudio_key = response.payload.data.decode("UTF-8")

In [14]:
# Only run this block for Vertex AI API
google_genai_client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION) # vertex
#google_genai_client = genai.Client(api_key=aistudio_key) # google ai studio

#google_genai_client

In [15]:
gemini_2_flash = 'gemini-2.0-flash-exp'

In [16]:
contents = [
    types.Content(
        role="user",
        parts=[types.Part.from_text(prompt)]
    )
]

In [17]:
generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
        category="HARM_CATEGORY_HATE_SPEECH",
        threshold="OFF"
    ),types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="OFF"
    ),types.SafetySetting(
        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
        threshold="OFF"
    ),types.SafetySetting(
        category="HARM_CATEGORY_HARASSMENT",
        threshold="OFF"
    )]
)

  

In [18]:
for chunk in google_genai_client.models.generate_content_stream(
    model = gemini_2_flash, 
    contents = contents, 
    config = generate_content_config,):
    print(chunk.text, end="")

RefreshError: ('invalid_scope: Invalid OAuth scope or ID token audience provided.', {'error': 'invalid_scope', 'error_description': 'Invalid OAuth scope or ID token audience provided.'})

In [30]:
response = google_genai_client.models.generate_content(
    model=gemini_2_flash, contents=prompt
)

In [31]:
IPython.display.Markdown(response.text)

A **mulligan** is a term used in various games, primarily card games and golf, to refer to a **second chance** to perform a particular action, usually a move or a draw, after an initial unsatisfactory attempt. The exact rules and context surrounding a mulligan vary depending on the game being played.

Here's a breakdown of how it's used in different contexts:

**1. Card Games (Most Commonly Poker, Magic: The Gathering, and Others):**

* **Meaning:** In most card games, a mulligan allows a player to discard their initial hand and draw a completely new hand. It's typically used when a player receives a hand that is considered unplayable or disadvantageous.
* **Rules:**
    * **Limited:** Usually, there are limits on how many mulligans a player can take in a game (e.g., one or two).
    * **Penalty:** Sometimes, there might be a small penalty associated with taking a mulligan, such as drawing one fewer card for the new hand.
    * **Starting Hands:** The need for a mulligan often stems from having a starting hand that lacks resources, doesn't have the right combination of cards, or is too slow to get going.
* **Purpose:** It's designed to make the game fairer and more enjoyable by mitigating the impact of bad luck in the initial draw.

**2. Golf:**

* **Meaning:** In golf, a mulligan is an informal second attempt at a shot, usually the first tee shot, after a poorly executed first attempt.
* **Rules:**
    * **Not Official:** A mulligan in golf is not part of the official rules of the game. It's a casual, friendly allowance, often used in social or practice rounds.
    * **Agreed Upon:** It's usually agreed upon by the players before the round begins whether or not mulligans will be allowed.
    * **First Tee Only:** It's most commonly applied to the first tee shot, but some groups might allow them in other situations.
* **Purpose:** It's meant to make the game more relaxed and forgiving, especially for beginner golfers.

**In Summary:**

A mulligan is a second chance, whether it's redrawing your hand in a card game or redoing a golf shot. It's a mechanic or practice aimed at reducing the impact of a poor start or initial disadvantage, ultimately making the game more enjoyable. It's crucial to understand the specific rules and context in which a mulligan is being used, as they vary considerably.
